<a href="https://colab.research.google.com/github/Taedriel/ZSL-v2/blob/wordEmbedding/WordsEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Kingston/ZSL-v2/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Kingston/ZSL-v2


In [2]:
!pip install transformers gensim wget

In [3]:
!git pull

remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/Taedriel/ZSL-v2
   48d25bc0..1751508f  wordEmbedding -> origin/wordEmbedding
Updating 48d25bc0..1751508f
Fast-forward
 converter.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [4]:
from converter import *
# ["king", "queen", "man", " woman", "splurgle", "pladonf"]
model = BERTModel()

model.import_tag_list("en-basic")
model.convert()
model.export("en-basic-Embedded.csv")

# model.computeCoSim()
# model.simBetween("cat", "dog")

man = model.get_embedding_of("man")[1]
woman = model.get_embedding_of("woman")[1]

king = model.get_embedding_of("king")[1]

totest = king.sub(man).add(woman)
print(model.get_nearest_embedding_of(totest, 3))

model.export_sim_matrix("sim_matrix.csv")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:Import finished : 850 elements imported.
INFO:root:Starting converting tokens...


['I', 'a', 'able', 'about', 'account', 'acid', 'across', 'act', 'addition', 'adjustment', 'advertisement', 'after', 'again', 'against', 'agreement', 'air', 'all', 'almost', 'among', 'amount', 'amusement', 'and', 'angle', 'angry', 'animal', 'answer', 'ant', 'any', 'apparatus', 'apple', 'approval', 'arch', 'argument', 'arm', 'army', 'art', 'as', 'at', 'attack', 'attempt', 'attention', 'attraction', 'authority', 'automatic', 'awake', 'baby', 'back', 'bad', 'bag', 'balance', 'ball', 'band', 'base', 'basin', 'basket', 'bath', 'be', 'beautiful', 'because', 'bed', 'bee', 'before', 'behaviour', 'belief', 'bell', 'bent', 'berry', 'between', 'bird', 'birth', 'bit', 'bite', 'bitter', 'black', 'blade', 'blood', 'blow', 'blue', 'board', 'boat', 'body', 'boiling', 'bone', 'book', 'boot', 'bottle', 'box', 'boy', 'brain', 'brake', 'branch', 'brass', 'bread', 'breath', 'brick', 'bridge', 'bright', 'broken', 'brother', 'brown', 'brush', 'bucket', 'building', 'bulb', 'burn', 'burst', 'business', 'but', '

INFO:root:10.0% completed
INFO:root:20.0% completed
INFO:root:30.0% completed
INFO:root:40.0% completed
INFO:root:50.0% completed
INFO:root:60.0% completed
INFO:root:70.0% completed
INFO:root:80.0% completed
INFO:root:90.0% completed


Exception: ignored